# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

In [7]:
system_prompt = "You are an AI expert that answers technical questions related to Generative AI. \
The answer should be short and smart, you can use a metaphore to explain a difficult problem. \
Provide a Python code example if it's relevant. Respond in markdown."

In [9]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

## GPT-4o-mini

In [13]:
# Get gpt-4o-mini to answer, with streaming

def stream_answer(user_question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [14]:
stream_answer(question)

The code snippet you've provided uses a Python feature called **generator expressions**, along with set comprehension. Let's break it down:

### Explanation

1. **`{book.get("author") for book in books if book.get("author")}`**:
   - This part iterates over a collection named `books`.
   - For each `book`, it attempts to get the value associated with the key `"author"`.
   - The condition `if book.get("author")` ensures that only books with an existing author (i.e., not `None` or an empty string) are included.
   - The result is a **set** of authors, meaning duplicates are removed.

2. **`yield from`**:
   - This keyword is used in generator functions to yield all items from an iterable (in this case, the set of authors).
   - It effectively "delegates" yielding to another iterable.

### Metaphor

Think of this code like a library's checkout desk where each book is a unique visitor. The checkout clerk (the generator) only hands the names of authors who are currently registered (exist in the book's data). If an author doesn't exist for a particular book, that book is skipped. When a registered author comes, he or she is announced (yielded) without duplicates.

### Python Example

Here's a small example illustrating its usage:

python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": None},
    {"title": "Book Four", "author": "Author A"},
]

# A generator function
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Using the generator
for author in get_unique_authors(books):
    print(author)


### Output

Author A
Author B


In this example, "Author A" only appears once because we collect authors in a set, demonstrating the intent of the original code!

## Llama 3.2

In [15]:
# Get Llama 3.2 to answer

def answear_question(question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    response = ollama.chat(model=MODEL_LLAMA, messages=messages)
    display(Markdown(response['message']['content']))

In [16]:
answear_question(question)

**Unpacking Nested Iterables**
================================

The given code snippet is using a feature of Python called "nested iterable unpacking" or "yield from". This allows us to iterate over the results of another generator expression.

Here's what it does:

*   `book.get("author")` attempts to retrieve the value associated with the key `"author"` from each book dictionary. If the key is missing, it returns `None`.
*   `{... for book in books if book.get("author")}` generates a sequence of values from these retrieved author names.
*   `yield from ...` then unpacks this sequence into individual yields, effectively "unfolding" the nested generator.

**Why?**
--------

This code is useful when you want to process each item in a list of books individually, but they are stored within another data structure (like a dictionary) where each book has its own author. It's like taking multiple baskets of fruit and unwrapping them to give you individual apples one by one.

**Example Use Case**
--------------------

Suppose we have a `books` list where each item is a dictionary with an `"author"` key:
```python
books = [
    {"title": "Book 1", "author": "John"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Jane"}
]

# Using the given code snippet to unpack author names
for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)
```
Output:
```
John
None
Jane
```

Note that `yield from` only works with iterable objects (like lists, generators, or dictionaries), not with non-iterable ones.

In [17]:
answear_question("What does 'yield from' in Python do?")

**Unpacking the Magic of `yield from`**
=====================================

`yield from` is a powerful feature in Python that allows you to "yield" multiple iterators one by one, making your code more efficient and easier to read.

Think of it like a librarian helping you access books from a shelf. Instead of retrieving all books at once, the librarian yields each book individually, allowing you to process them sequentially.

**Code Example**
```python
def flatten(iterable):
    """Yield elements from multiple iterators"""
    for item in iterable:
        if isinstance(item, Iterable) and not isinstance(item, str):
            yield from flatten(item)
        else:
            yield item

# Usage example
numbers = [1, 2]
letters = ['a', 'b']

for elem in flatten([numbers, letters]):
    print(elem)
```
Output:
```
1
2
a
b
```
In this example, `flatten` yields elements from the input iterable, recursively yielding from any sub-iterables. The `yield from` statement is used to delegate to another iterator.

**When to use `yield from`?**

Use `yield from` when you need to:

* Handle multiple iterators or sequences
* Reduce memory usage by processing data in chunks
* Simplify your code and make it more readable

Remember, `yield from` is not just a shortcut; it's a powerful tool for writing efficient and elegant generators!